In [ ]:
%%HTML
<!-- Mejorar visualización en proyector -->
<style>
.rendered_html {font-size: 1.2em; line-height: 150%;}
div.prompt {min-width: 0ex; padding: 0px;}
.container {width:95% !important;}
</style>

In [1]:
%autosave 0
%matplotlib notebook
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import display
from matplotlib import animation
import ipywidgets as widgets
from functools import partial
slider_layout = widgets.Layout(width='600px', height='20px')
slider_style = {'description_width': 'initial'}
IntSlider_nice = partial(widgets.IntSlider, style=slider_style, layout=slider_layout, continuous_update=False)
SelSlider_nice = partial(widgets.SelectionSlider, style=slider_style, layout=slider_layout, continuous_update=False)

Autosave disabled


# Estadística

> Disciplina científicaEs una colección de métodos que utilizan datos para responder preguntas

> Busca entender fenómenos complejos a partir de observaciones parciales e inciertas

> Inferir la probabilidad de un fenomeno aleatorio basándonos en observaciones (datos)

> Disciplina científica dedicada al desarrollo y estudio de métodos para recopilar, analizar e interpretar a partir de datos

> Aprendizaje estadística $\equiv$ Machine Learning

Conceptos fundamentales

- Incerteza
- Variación


https://machinelearningmastery.com/what-is-statistics/

https://www.sscc.edu/home/jdavidso/MathAdvising/AboutStatistics.html

https://elearningindustry.com/stats-101-need-know-statistics

https://towardsdatascience.com/statistics-descriptive-and-inferential-63661eb13bb5

- Fundamentos: Ley de los grandes números y teorema central del límite
- Números aleatorios
- Métodos no parámetros: Histograma y KDE
- Métodos parámetricos